<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

[https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches](https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01)


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/Falcon9\_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing\_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/falcon9-launches-wiki.png)


## Objectives

Web scrap Falcon 9 launch records with `BeautifulSoup`:

*   Extract a Falcon 9 launch records HTML table from Wikipedia
*   Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [ ]:
!pip3 install beautifulsoup4
!pip3 install requests

In [1]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [2]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [3]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [4]:
# use requests.get() method with the provided static_url
# assign the response to a object
r=requests.get(static_url)

In [8]:
r.headers['content-type']

'text/html; charset=UTF-8'

Create a `BeautifulSoup` object from the HTML `response`


In [9]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
datacontent=r.text
soup=BeautifulSoup(datacontent,"html5lib")

Print the page title to verify if the `BeautifulSoup` object was created properly


In [10]:
# Use soup.title attribute
title1=soup.find('title')
print(title1)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [12]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables=soup.find_all('table')

In [13]:
type(html_tables)

bs4.element.ResultSet

## <font color=darkblue>  The following codes are for practice purpose </font>

In [70]:
for ik,jk in enumerate(html_tables):
    if 'falcon' in str(jk):
        ind1=ik
        print(f'find {ik}')

find 6


In [71]:
html_tables[ind1]

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a><sup class="reference" id="cite_ref-booster_11-4"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-4"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">29
</th>
<td>

In [72]:
table6=pd.read_html(str(html_tables[ind1]),flavor='bs4')

In [75]:
table6[0]

,Flight No.,Date andtime (UTC),"Version,Booster[b]",Launch site,Payload[c],Payload mass,Orbit,Customer,Launchoutcome,Boosterlanding
0,29,"14 January 2017,17:54",F9 FTB1029.1[151],"VAFB,SLC-4E",Iridium NEXT-1(10 satellites)[152][153],"9,600 kg (21,200 lb)",Polar LEO,Iridium Communications,Success,Success[154](drone ship)
1,29,Return-to-flight mission after the loss of Amo...,Return-to-flight mission after the loss of Amo...,Return-to-flight mission after the loss of Amo...,Return-to-flight mission after the loss of Amo...,Return-to-flight mission after the loss of Amo...,Return-to-flight mission after the loss of Amo...,Return-to-flight mission after the loss of Amo...,Return-to-flight mission after the loss of Amo...,Return-to-flight mission after the loss of Amo...
2,30,"19 February 2017,14:39",F9 FTB1031.1[8],"KSC,LC-39A",SpaceX CRS-10[141](Dragon C112.1),"2,490 kg (5,490 lb)[160]",LEO (ISS),NASA (CRS),Success,Success(ground pad)
3,30,First Falcon 9 flight from the historic LC-39A...,First Falcon 9 flight from the historic LC-39A...,First Falcon 9 flight from the historic LC-39A...,First Falcon 9 flight from the historic LC-39A...,First Falcon 9 flight from the historic LC-39A...,First Falcon 9 flight from the historic LC-39A...,First Falcon 9 flight from the historic LC-39A...,First Falcon 9 flight from the historic LC-39A...,First Falcon 9 flight from the historic LC-39A...
4,31,"16 March 2017,06:00",F9 FTB1030.1[163],"KSC,LC-39A",EchoStar 23,"5,600 kg (12,300 lb)[164]",GTO,EchoStar,Success,No attempt[165]
5,31,First uncrewed non-station launch from LC-39A ...,First uncrewed non-station launch from LC-39A ...,First uncrewed non-station launch from LC-39A ...,First uncrewed non-station launch from LC-39A ...,First uncrewed non-station launch from LC-39A ...,First uncrewed non-station launch from LC-39A ...,First uncrewed non-station launch from LC-39A ...,First uncrewed non-station launch from LC-39A ...,First uncrewed non-station launch from LC-39A ...
6,32,"30 March 2017,22:27",F9 FT ♺B1021.2[115],"KSC,LC-39A",SES-10[109][168],"5,300 kg (11,700 lb)[169]",GTO,SES,Success[170],Success(drone ship)
7,32,"First payload to fly on a reused first stage, ...","First payload to fly on a reused first stage, ...","First payload to fly on a reused first stage, ...","First payload to fly on a reused first stage, ...","First payload to fly on a reused first stage, ...","First payload to fly on a reused first stage, ...","First payload to fly on a reused first stage, ...","First payload to fly on a reused first stage, ...","First payload to fly on a reused first stage, ..."
8,33,"1 May 2017,11:15",F9 FTB1032.1[129],"KSC,LC-39A",NROL-76[174],Classified,LEO[175],NRO,Success,Success(ground pad)
9,33,First launch under SpaceX's 2015 certification...,First launch under SpaceX's 2015 certification...,First launch under SpaceX's 2015 certification...,First launch under SpaceX's 2015 certification...,First launch under SpaceX's 2015 certification...,First launch under SpaceX's 2015 certification...,First launch under SpaceX's 2015 certification...,First launch under SpaceX's 2015 certification...,First launch under SpaceX's 2015 certification...


In [77]:
html_table6=html_tables[6]

In [17]:
html_tables[0]

<table class="multicol" role="presentation" style="border-collapse: collapse; padding: 0; border: 0; background:transparent; width:100%;">

<tbody><tr>
<td style="text-align: left; vertical-align: top;">
<h3><span class="mw-headline" id="Rocket_configurations">Rocket configurations</span></h3>
<div class="chart noresize" style="margin-top:1em;max-width:420px;">
<div style="position:relative;min-height:320px;min-width:420px;max-width:420px;">
<div style="float:right;position:relative;min-height:240px;min-width:320px;max-width:320px;border-left:1px black solid;border-bottom:1px black solid;">
<div style="position:absolute;left:3px;top:224px;height:15px;min-width:18px;max-width:18px;background-color:LightSteelBlue;-webkit-print-color-adjust:exact;border:1px solid LightSteelBlue;border-bottom:none;overflow:hidden;" title="[[Falcon 9 v1.0]]: 2"></div>
<div style="position:absolute;left:55px;top:224px;height:15px;min-width:18px;max-width:18px;background-color:LightSteelBlue;-webkit-print-col

Starting from the third table is our target table contains the actual launch records.


## <font color=darkblue>  Practice TimeOut </font>

In [18]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [27]:
type(first_launch_table)

bs4.element.Tag

In [178]:
table3=pd.read_html(str(first_launch_table),flavor=None) # use read_html to convert table to dataframe

In [179]:
len(table3)

1

In [180]:
table3[0].columns

Index(['Flight No.', 'Date andtime (UTC)', 'Version,Booster [b]',
       'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer',
       'Launchoutcome', 'Boosterlanding'],
      dtype='object')

In [181]:
column_names_pd = table3[0].columns.to_list()

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


In [182]:
row_tr1=first_launch_table.find_all('tr')

In [189]:
row_tr1[0].th.string

'Flight No.\n'

In [190]:
head1=row_tr1[0].find_all('th')

In [203]:
head1[2].a.extract()

<a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a>

In [210]:
extract_column_from_header(row_tr1[0])

TypeError: sequence item 1: expected str instance, Tag found

In [211]:
column_names_pd

['Flight No.',
 'Date andtime (UTC)',
 'Version,Booster [b]',
 'Launch site',
 'Payload[c]',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launchoutcome',
 'Boosterlanding']

Check the extracted column names


In [207]:
print(column_names)

[]


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [212]:
launch_dict= dict.fromkeys(column_names_pd)

In [213]:
launch_dict

{'Flight No.': None,
 'Date andtime (UTC)': None,
 'Version,Booster [b]': None,
 'Launch site': None,
 'Payload[c]': None,
 'Payload mass': None,
 'Orbit': None,
 'Customer': None,
 'Launchoutcome': None,
 'Boosterlanding': None}

In [216]:
launch_dict['Version,Booster [b]'].append('v1.1')

AttributeError: 'NoneType' object has no attribute 'append'

In [220]:
launch_dict.keys()

dict_keys(['Flight No.', 'Date andtime (UTC)', 'Version,Booster [b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding'])

In [222]:
for key in launch_dict.keys():
    launch_dict[key]=[]

In [223]:
launch_dict

{'Flight No.': [],
 'Date andtime (UTC)': [],
 'Version,Booster [b]': [],
 'Launch site': [],
 'Payload[c]': [],
 'Payload mass': [],
 'Orbit': [],
 'Customer': [],
 'Launchoutcome': [],
 'Boosterlanding': []}

In [227]:
del launch_dict['Date andtime (UTC)']

In [230]:
launch_dict['Date']=[] 
launch_dict['Time']=[]

## <font color=red> This original code not used. Replaced by the above lines </font>
launch_dict= dict.fromkeys(column_names_pd)

### Remove an irrelvant column
del launch_dict['Date andtime (UTC)']

#### Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
#### Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [36]:
tableWiki=soup.find_all("wikitable plainrowheaders collapsible")

In [39]:
len(tableWiki)

0

In [38]:
len(html_tables)

24

In [40]:
table5=html_tables[4]

In [58]:
type(table5.find_all('th'))

bs4.element.ResultSet

In [57]:
type(table5.find_all('th')[2])

bs4.element.Tag

In [56]:
if 'scope' in str(table5.find_all('th')[2]):
    print('scope')

scope


In [128]:
rows=html_tables[4].find_all('tr')

In [129]:
pd.read_html(str(html_tables[4]),flavor='bs4')[0]

,Flight No.,Date andtime (UTC),"Version,Booster[b]",Launch site,Payload[c],Payload mass,Orbit,Customer,Launchoutcome,Boosterlanding
0,14,"10 January 2015,09:47[67]",F9 v1.1B1012[8],"Cape Canaveral,LC-40",SpaceX CRS-5[68](Dragon C107),"2,395 kg (5,280 lb)[69]",LEO (ISS),NASA (CRS),Success[70],Failure (drone ship)
1,14,"Following second-stage separation, SpaceX atte...","Following second-stage separation, SpaceX atte...","Following second-stage separation, SpaceX atte...","Following second-stage separation, SpaceX atte...","Following second-stage separation, SpaceX atte...","Following second-stage separation, SpaceX atte...","Following second-stage separation, SpaceX atte...","Following second-stage separation, SpaceX atte...","Following second-stage separation, SpaceX atte..."
2,15,"11 February 2015,23:03[72]",F9 v1.1B1013[8],"Cape Canaveral,LC-40",DSCOVR[68][73],"570 kg (1,260 lb)",HEO(Sun–Earth L1 insertion),USAF NASA NOAA,Success,Controlled(ocean)[d]
3,15,First launch under USAF's OSP 3 launch contrac...,First launch under USAF's OSP 3 launch contrac...,First launch under USAF's OSP 3 launch contrac...,First launch under USAF's OSP 3 launch contrac...,First launch under USAF's OSP 3 launch contrac...,First launch under USAF's OSP 3 launch contrac...,First launch under USAF's OSP 3 launch contrac...,First launch under USAF's OSP 3 launch contrac...,First launch under USAF's OSP 3 launch contrac...
4,16,"2 March 2015,03:50[21][76]",F9 v1.1B1014[8],"Cape Canaveral,LC-40",ABS-3A Eutelsat 115 West B[68],"4,159 kg (9,169 lb)",GTO,ABS Eutelsat,Success,No attempt[77]
5,16,The launch was Boeing's first conjoined launch...,The launch was Boeing's first conjoined launch...,The launch was Boeing's first conjoined launch...,The launch was Boeing's first conjoined launch...,The launch was Boeing's first conjoined launch...,The launch was Boeing's first conjoined launch...,The launch was Boeing's first conjoined launch...,The launch was Boeing's first conjoined launch...,The launch was Boeing's first conjoined launch...
6,17,"14 April 2015,20:10[21]",F9 v1.1B1015[8],"Cape Canaveral,LC-40",SpaceX CRS-6[68](Dragon C108.1),"1,898 kg (4,184 lb)[82]",LEO (ISS),NASA (CRS),Success,Failure[83](drone ship)
7,17,"After second-stage separation, a controlled-de...","After second-stage separation, a controlled-de...","After second-stage separation, a controlled-de...","After second-stage separation, a controlled-de...","After second-stage separation, a controlled-de...","After second-stage separation, a controlled-de...","After second-stage separation, a controlled-de...","After second-stage separation, a controlled-de...","After second-stage separation, a controlled-de..."
8,18,"27 April 2015,23:03[86]",F9 v1.1B1016[8],"Cape Canaveral,LC-40",TürkmenÄlem 52°E / MonacoSAT[68][87],"4,707 kg (10,377 lb)",GTO,Turkmenistan NationalSpace Agency[88],Success,No attempt[89]
9,18,Original intended launch was delayed over a mo...,Original intended launch was delayed over a mo...,Original intended launch was delayed over a mo...,Original intended launch was delayed over a mo...,Original intended launch was delayed over a mo...,Original intended launch was delayed over a mo...,Original intended launch was delayed over a mo...,Original intended launch was delayed over a mo...,Original intended launch was delayed over a mo...


In [130]:
rows[0].th.string

'Flight No.\n'

In [131]:
rows[0].th.string.strip()

'Flight No.'

In [132]:
rows[1].th.string.strip().isdigit()

True

In [146]:
col2=rows[3].find_all('td')

In [163]:
col2[1].a.string

'F9 v1.1'

In [240]:
col2[6].a

<a href="/wiki/United_States_Air_Force" title="United States Air Force">USAF</a>

In [174]:
rows[3]

<tr>
<th rowspan="2" scope="row" style="text-align:center;">15
</th>
<td>11 February 2015,<br/>23:03<sup class="reference" id="cite_ref-79"><a href="#cite_note-79">[72]</a></sup>
</td>
<td><a href="/wiki/Falcon_9_v1.1" title="Falcon 9 v1.1">F9 v1.1</a><br/>B1013<sup class="reference" id="cite_ref-block_numbers_14-9"><a href="#cite_note-block_numbers-14">[8]</a></sup>
</td>
<td><a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">Cape Canaveral</a>,<br/><a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">LC-40</a>
</td>
<td><a class="mw-redirect" href="/wiki/DSCOVR" title="DSCOVR">DSCOVR</a><sup class="reference" id="cite_ref-sxManifest20130731_75-1"><a href="#cite_note-sxManifest20130731-75">[68]</a></sup><sup class="reference" id="cite_ref-80"><a href="#cite_note-80">[73]</a></sup>
</td>
<td>570 kg (1,260 lb)
</td>
<td><a href="/wiki/High_Earth_orbit" title="High Earth orbit">HEO</a><br/>(<a hre

In [139]:
len(col2)

1

In [100]:
type(soup('table')) # same as soup.find_all('table')

bs4.element.ResultSet

In [234]:
list(launch_dict.keys())

['Flight No.',
 'Version,Booster [b]',
 'Launch site',
 'Payload[c]',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launchoutcome',
 'Boosterlanding',
 'Date',
 'Time']

In [243]:
for key in launch_dict.keys():
    launch_dict[key]=[]
# above line initializes the dictionary when calling this code
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            key=list(launch_dict.keys())[0]
            launch_dict[key].append(flight_number)
            print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            launch_dict['Time'].append(time)                                 
            print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)
            key=list(launch_dict.keys())[1]
            launch_dict[key].append(bv)                             
        
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            print(launch_site)
            key=list(launch_dict.keys())[2]
            launch_dict[key].append(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            print(payload)
            key=list(launch_dict.keys())[3]
            launch_dict[key].append(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            print(payload)
            key=list(launch_dict.keys())[4]
            launch_dict[key].append(payload_mass)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            key=list(launch_dict.keys())[5]
            launch_dict[key].append(orbit)
            print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            if row[6].a:             # several rows have no hyperlink. if it has hyperlink (row.a), use row.a.string
                customer = row[6].a.string
            else:
                customer = row[6].string
            print(customer)
            key=list(launch_dict.keys())[6]
            launch_dict[key].append(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            print(launch_outcome)
            key=list(launch_dict.keys())[7]
            launch_dict[key].append(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            print(booster_landing)
            key=list(launch_dict.keys())[8]
            launch_dict[key].append(booster_landing)
            

1
4 June 2010
18:45
F9 v1.0B0003.1
CCAFS
Dragon Spacecraft Qualification Unit
Dragon Spacecraft Qualification Unit
LEO
SpaceX
Success

Failure
2
8 December 2010
15:43
F9 v1.0B0004.1
CCAFS
Dragon
Dragon
LEO
NASA
Success
Failure
3
22 May 2012
07:44
F9 v1.0B0005.1
CCAFS
Dragon
Dragon
LEO
NASA
Success
No attempt

4
8 October 2012
00:35
F9 v1.0B0006.1
CCAFS
SpaceX CRS-1
SpaceX CRS-1
LEO
NASA
Success

No attempt
5
1 March 2013
15:10
F9 v1.0B0007.1
CCAFS
SpaceX CRS-2
SpaceX CRS-2
LEO
NASA
Success

No attempt

6
29 September 2013
16:00
F9 v1.1B1003
VAFB
CASSIOPE
CASSIOPE
Polar orbit
MDA
Success
Uncontrolled
7
3 December 2013
22:41
F9 v1.1
CCAFS
SES-8
SES-8
GTO
SES
Success
No attempt
8
6 January 2014
22:06
F9 v1.1
CCAFS
Thaicom 6
Thaicom 6
GTO
Thaicom
Success
No attempt
9
18 April 2014
19:25
F9 v1.1
Cape Canaveral
SpaceX CRS-3
SpaceX CRS-3
LEO
NASA
Success

Controlled
10
14 July 2014
15:15
F9 v1.1
Cape Canaveral
Orbcomm-OG2
Orbcomm-OG2
LEO
Orbcomm
Success
Controlled
11
5 August 2014
08:00
F9 v1

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [244]:
df=pd.DataFrame(launch_dict)

In [245]:
df

,Flight No.,"Version,Booster [b]",Launch site,Payload[c],Payload mass,Orbit,Customer,Launchoutcome,Boosterlanding,Date,Time
0,1,F9 v1.0B0003.1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,Failure,4 June 2010,18:45
1,2,F9 v1.0B0004.1,CCAFS,Dragon,0,LEO,NASA,Success,Failure,8 December 2010,15:43
2,3,F9 v1.0B0005.1,CCAFS,Dragon,525 kg,LEO,NASA,Success,No attempt\n,22 May 2012,07:44
3,4,F9 v1.0B0006.1,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,No attempt,8 October 2012,00:35
4,5,F9 v1.0B0007.1,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,No attempt\n,1 March 2013,15:10
...,...,...,...,...,...,...,...,...,...,...,...
116,117,F9 B5B1051.10,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,Success,9 May 2021,06:42
117,118,F9 B5B1058.8,KSC,Starlink,"~14,000 kg",LEO,SpaceX,Success\n,Success,15 May 2021,22:56
118,119,F9 B5B1063.2,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,Success,26 May 2021,18:59
119,120,F9 B5B1067.1,KSC,SpaceX CRS-22,"3,328 kg",LEO,NASA,Success\n,Success,3 June 2021,17:29


In [246]:
df.to_csv('spacex_web_scraped.csv', index=False)

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab.

Following labs will be using a provided dataset to make each lab independent.


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates               |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
